The instructions below are based on the RStudio keras [tutorial](https://keras.rstudio.com/articles/tutorial_basic_classification.html#explore-the-data) for classifying a simple Fashion MNIST sample.

The Fashion MNIST sample is pretty much equivalent to the classic MNIST example we just saw, however, instead of classifying the images into categories from 0 to 9 we want to classify images by the type of clothing they are.

**The goal of this notebook is to get you to try your hand at a simple ANN yourself.
Please refer back to the MNIST example to help you along with this notebook.**

The Fashion MNIST dataset contains 70,000 grayscale images in 10 categories. The images show individual articles of clothing at low resolution (28 by 28 pixels), as seen here:

![sample](https://keras.rstudio.com/articles/images/fashion_mnist.png)
(Image by Zalando, MIT License)

In [ ]:
library(keras)
use_condaenv("r-tensorflow")
use_session_with_seed(7)
options(keras.view_metrics = TRUE)

In [ ]:
#############################################
# Prepping the data
#############################################

# Load the images
fashion_mnist <- dataset_fashion_mnist()

c(train_images, train_labels) %<-% fashion_mnist$train
c(test_images, test_labels) %<-% fashion_mnist$test

class_names = c('T-shirt/top',
                'Trouser',
                'Pullover',
                'Dress',
                'Coat', 
                'Sandal',
                'Shirt',
                'Sneaker',
                'Bag',
                'Ankle boot')


In [ ]:
dim(train_images)

In [ ]:
dim(train_labels)

In [ ]:
train_labels[1:20]

## Preprocess the data

The data must be preprocessed before training the network. If you inspect the first image in the training set, you will see that the pixel values fall in the range of 0 to 255.

We scale these values to a range of 0 to 1 before feeding to the neural network model. For this, we simply divide by 255.


In [ ]:
# Plotting a couple of samples
library(tidyr)
library(ggplot2)

i = 1  # <=====   Change me to see the different digits and their labels
img <- train_images[i, , ]
img <- t(apply(img, 2, rev))


options(repr.plot.width=2, repr.plot.height=2)
par(oma=c(0,0,0,0), mar=c(0,0,2,0))
image(img,col = gray((0:255)/255), asp=1, axes=FALSE)
title(main = paste("Type: ",class_names[train_labels[i]+1]), cex.main=0.7)

In [ ]:
# It’s important that the training set and the test set are preprocessed in the same way
# We want our training images (and tes images) to be between 0 and 1

train_images <- # Scale the images (ie divide by 255) 
test_images <- # Scale the images (ie divide by 255) 

## Building the Model

Building the neural network requires configuring the layers of the model, then compiling the model.

### Setup the layers

The basic building block of a neural network is the layer. Layers extract representations from the data fed into them. And, hopefully, these representations are more meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, like layer_dense, have parameters that are learned during training.


For this network the first layer, layer_flatten, transforms the format of the images from a 2d-array (of 28 by 28 pixels), to a 1d-array of 28 * 28 = 784 pixels. Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn; it only reformats the data.

After the pixels are flattened, the network consists of a sequence of two dense layers. These are densely-connected, or fully-connected, neural layers. The first dense layer has 128 nodes (or neurons). The second (and last) layer is a 10-node softmax layer —
this returns an array of 10 probability scores that sum to 1. Each node contains a score that indicates the probability that the current image belongs to one of the 10 digit classes.


In [ ]:
model <- # load the sequential model
model %>%
    # The first layer flattens the 2D images to 1D vectors and tells the network the input shape
    layer_flatten(input_shape = c(28, 28)) %>% 
    # The next two layers need the units (how many neurons) and activation functions
    layer_dense(units = , activation = ) %>% 
    layer_dense(units = , activation = ) # remember we want to classify 10 categories

### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model’s compile step:

-    Loss function — This measures how accurate the model is during training. We want to minimize this function to “steer” the model in the right direction.
-    Optimizer — This is how the model is updated based on the data it sees and its loss function.
-    Metrics —Used to monitor the training and testing steps. The following example uses accuracy, the fraction of the images that are correctly classified.


In [ ]:
model %>% compile(
  optimizer = 'adam', 
  loss = 'sparse_categorical_crossentropy',
  metrics = c('accuracy')
)

summary(model)

### Train the model

Training the neural network model requires the following steps:

-    Feed the training data to the model — in this example, the train_images and train_labels arrays.
-    The model learns to associate images and labels.
-    We ask the model to make predictions about a test set — in this example, the test_images array. We verify that the predictions match the labels from the test_labels array.

To start training, call the fit method — the model is “fit” to the training data:

In [ ]:
#############################################
# Training and evaluation
#############################################

history <- model %>% fit(
  # insert the training data here
  epochs = 30, batch_size = 128,
  validation_split = 0.2
)


history


options(repr.plot.width=6, repr.plot.height=4)
plot(history)

### Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
#evaluating the model
model %>% evaluate(# insert the test data here)

### Make predictions

With the model trained, we can use it to make predictions about some images.

In [ ]:
predictions <- model %>% predict(# enter the test images here)

Let’s plot several images with their predictions. Correct prediction labels are green and incorrect prediction labels are red.

In [ ]:
par(mfcol=c(5,5))
par(mar=c(0, 0, 1.5, 0), xaxs='i', yaxs='i')
for (i in 1:25) { 
  img <- test_images[i, , ]
  img <- t(apply(img, 2, rev)) 
  # subtract 1 as labels go from 0 to 9
  predicted_label <- which.max(predictions[i, ]) - 1
  true_label <- test_labels[i]
  if (predicted_label == true_label) {
    color <- '#008800' 
  } else {
    color <- '#bb0000'
  }
  image(1:28, 1:28, img, col = gray((0:255)/255), xaxt = 'n', yaxt = 'n',
        main = paste0(class_names[predicted_label + 1], " (",
                      class_names[true_label + 1], ")"),
        col.main = color)
}